In [1]:
from sympy import *
from sympy.matrices import Matrix, eye
# from mpmath import radians, degrees
import numpy as np
# import matplotlib as mpl
import matplotlib.pyplot as plt
# from __future__ import print_function
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets.widgets import *
# from d

In [2]:
def col(input_tuple, h=0):
    matrixArg = [[item] for item in input_tuple]
    if h: matrixArg.append([1])
    return Matrix(matrixArg)

def composeRotations(rotation_list=[]):
    "rotation_list contains tuples of form (axis, radian_angle), where axis is an element of {'x', 'y', 'z'}"
    rMat = eye(3)
    def rot_x(q):
        R_x = Matrix([[1, 0, 0],
                      [0, cos(q), -sin(q)],
                      [0, sin(q), cos(q)]])
        return R_x

    def rot_y(q):
        R_y = Matrix([[cos(q), 0, sin(q)],
                      [0, 1, 0],
                      [-sin(q), 0, cos(q)]])
        return R_y

    def rot_z(q):
        R_z = Matrix([[cos(q), -sin(q), 0],
                      [sin(q), cos(q), 0],
                      [0, 0, 1]])
        return R_z

    for rotation in rotation_list:
        rotation_axis, rotation_angle = rotation[0], rotation[1]
        rr = eval('rot_{}({})'.format(rotation_axis, rotation_angle))
        rMat = rr * rMat
    return rMat


def TF_matrix(rotation_matrix, translation=(0, 0, 0)):
    "rotation_list formatted for composeRotations(),"
    TFmat = rotation_matrix.row_join(col(translation))
    TFmat = TFmat.col_join(Matrix([[0, 0, 0, 1]]))
    return TFmat


def DH_TF(alpha, a, d, q, correction_matrix=None):
    R_corr = correction_matrix


    if not correction_matrix: R_corr = eye(4)
    # alpha, a, d, q = twist_angle, link_length, joint_angle, link_offset
    subT_x = TF_matrix(composeRotations([('x', alpha)]), (a, 0, 0))
    subT_z = TF_matrix(composeRotations([('z', q)]), (0, 0, d))
    return subT_x * subT_z * R_corr


def rotation_matrix_from(DH_TF_mat):
    return DH_TF_mat[0:3, 0:3]


def translation_column_from(DH_TF_mat):
    return DH_TF_mat[0:3, 3]

def vector_cross_angle(first_vector, second_vector):
    # The vector arguments should be Matrix objects with three elements
    unit_1 = first_vector.normalized()
    unit_2 = second_vector.normalized()
    cross_vector = unit_1.cross(unit_2)
    return N(asin(cross_vector.norm()))

def vector_cross_unit(first_vector, second_vector):
    # The vector arguments should be Matrix objects with three elements
    unit_1 = first_vector.normalized()
    unit_2 = second_vector.normalized()
    cross_vector = unit_1.cross(unit_2)
    return cross_vector.normalized()

In [3]:
joint_decomp = {'urdf': {},
                'DH_strict': {},
                'DH_actual': {}
                }

joint_decomp['urdf']['x'] = [0, 0, 0.35, 0.35, 1.31, 1.85, 2.043, 2.153, 2.303]
joint_decomp['urdf']['y'] = [0, 0, 0, 0, 0, 0, 0, 0, 0]
joint_decomp['urdf']['z'] = [0, 0.33, 0.75, 2.0, 1.946, 1.946, 1.946, 1.946, 1.946]

joint_decomp['DH_strict']['x'] = [0, 0, 0.35, 0.35, 1.85, 1.85, 1.85, 2.153, 2.303]
joint_decomp['DH_strict']['y'] = [0, 0, 0, 0, 0, 0, 0, 0, 0]
joint_decomp['DH_strict']['z'] = [0, 0.75, 0.75, 2.0, 1.946, 1.946, 1.946, 1.946, 1.946]

joint_decomp['DH_actual']['x'] = [0, 0, 0.35, 0.35, 1.31, 1.85, 2.043, 2.153, 2.303]
joint_decomp['DH_actual']['y'] = [0, 0, 0, 0, 0, 0, 0, 0, 0]
joint_decomp['DH_actual']['z'] = [0, 0.75, 0.75, 2.0, 1.946, 1.946, 1.946, 1.946, 1.946]

In [4]:
relative_joints = {0: {'urdf': (0, 0, 0), # base_link origin
                       'dh':   (0, 0, 0)},
                   
                   1: {'urdf': (0, 0, 0.33),
                       'dh':   (0, 0, 0.75)},
                   
                   2: {'urdf': (0.35, 0, 0.42),
                       'dh':   (0.35, 0, 0)},
                   
                   3: {'urdf': (0, 0, 1.25),
                       'dh':   (0, 0, 1.25)},
                   
                   4: {'urdf': (0.96, 0, -0.054),
                       'dh':   (1.5, 0, -0.054)},
                   
                   5: {'urdf': (0.54, 0, 0),
                       'dh':   (0, 0, 0)},
                   
                   6: {'urdf': (0, 0, 0),
                       'dh':   (0, 0, 0)},
                   
                   7: {'urdf': (0.11, 0, 0),
                       'dh':   (0.11, 0, 0)}, # gripper_joint origin
                   
                   8: {'urdf': (0, 0, 0),
                       'dh':   (0, 0, 0)}  # extended finger target origin
}
                   
joints = {0: {'urdf': (0, 0, 0)}, # base_link origin
          1: {'urdf': (0, 0, 0.33)},
          2: {'urdf': (0.35, 0, 0.75)},
          3: {'urdf': (0.35, 0, 2.00)},
          4: {'urdf': (1.31, 0, 1.946)},
          5: {'urdf': (1.85, 0, 1.946)},
          6: {'urdf': (2.043, 0, 1.946)},
          7: {'urdf': (2.153, 0, 1.946)}, # gripper_joint origin
          8: {'urdf': (2.303, 0, 1.946)}} # extended finger target origin

zero_pose_joint_axes = {1: 'z',
                        2: 'y',
                        3: 'y',
                        4: 'x',
                        5: 'y',
                        6: 'x'
                        }

def link_vector(link_number, convention, unit=False):
    if link_number not in range(0, 8):
        print('invalid link index!')
        return None
    og_vect, targ_vect = col(joints[link_number][convention]), col(joints[link_number+1][convention])
    if unit:
        return (targ_vect - og_vect).normalized()
    return targ_vect - og_vect

def lists_from_points(joint_convention):
    x, y, z = [], [], []
    for n in range(0, 9):
        x.append(joints[n][joint_convention][0])
        y.append(joints[n][joint_convention][1])
        z.append(joints[n][joint_convention][2])
    return x, y, z

# def transform

def render_pose(q_list, convention):
    temp_joints = [col(joints[i][convention],h=1) for i in range(0, 9)]
    # for each joint q, calculate a rotation matrix to apply to all joints after q, about Z_q.
    for i in range(1, 7):
#         print('q{}'.format(i))
        xlate = (temp_joints[i][0], temp_joints[i][1], temp_joints[i][2])
        
        move_from_og = TF_matrix(eye(3), translation=xlate)
        
        R = TF_matrix(composeRotations([(zero_pose_joint_axes[i], q_list[i-1])]))
        
        TF = move_from_og.inv() * R * move_from_og
        
        for j in range(0, len(temp_joints[i::])):
#             print('rotating joint {}'.format(i+j))
            temp_joints[i+j] = N(TF * temp_joints[i+j])
        
    x, y, z = [], [], []
    for n in range(0, 9):
        x.append(temp_joints[n][0])
        y.append(temp_joints[n][1])
        z.append(temp_joints[n][2])
        
    # ax.legend()

    return x, y, z

In [5]:
q_list = [0, 0, 0, 0, 0, 0]

@interact
def get_q_values(q1=0, q2=0, q3=0, q4=0, q5=0, q6=0):
    global q_list
    q_list[0] = q1
    q_list[1] = q2
    q_list[2] = q3
    q_list[3] = q4
    q_list[4] = q5
    q_list[5] = q6
    print(q_list)    

interactive(children=(IntSlider(value=0, description='q1', max=1), IntSlider(value=0, description='q2', max=1), IntSlider(value=0, description='q3', max=1), IntSlider(value=0, description='q4', max=1), IntSlider(value=0, description='q5', max=1), IntSlider(value=0, description='q6', max=1), Output()), _dom_classes=('widget-interact',))

In [6]:
x, y, z = render_pose(get_q_values(), 'urdf')

def livedraw(x, y, z):
    %matplotlib inline
    fig = plt.figure()
    ax = fig.gca(projection='3d')

    ax.plot(x, y, z, label='urdf_joint_positions')
    
    ax.scatter(x, y, z, marker='o')

    plt.show()

# x, y, z

[0, 0, 0, 0, 0, 0]


TypeError: 'NoneType' object is not subscriptable

In [15]:
interact?


In [ ]:
# def fk_orientation(q_list):
# help(Matrix.inv)

In [7]:
%matplotlib
# mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')

# x = joint_decomp['urdf']['x']
# y = joint_decomp['urdf']['y']
# z = joint_decomp['urdf']['z']

ax.plot(x, y, z, label='urdf_joint_positions')
finger_center = x.pop()
ax.scatter(x, y, z, marker='o')
# ax.legend()

plt.show()

Using matplotlib backend: Qt5Agg


NameError: name 'x' is not defined

In [ ]:
import plotly.plotly as py
import numpy as np

data = [dict(
        visible = False,
        line=dict(color='00CED1', width=6),
        name = '𝜈 = '+str(step),
        x = np.arange(0,10,0.01),
        y = np.sin(step*np.arange(0,10,0.01))) for step in np.arange(0,5,0.1)]
data[10]['visible'] = True

steps = []
for i in range(len(data)):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data)],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 10,
    currentvalue = {"prefix": "Frequency: "},
    pad = {"t": 50},
    steps = steps
)]

layout = dict(sliders=sliders)
fig = dict(data=data, layout=layout)

py.plot(fig, filename='Sine Wave Slider')

In [ ]:
dir(py)

In [ ]:
help(py.get_config);

In [ ]:
# py.sign_in('General-Sax', '8AWT8Kb7EEH3tgqBTnJm')
py.get_config()

In [22]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python

In [1]:
%%dot
digraph G {
    rankdir=RL;
    theta_1 -> wrist_x
    theta_1 -> wrist_y
    wrist_x -> wrist_center
    wrist_y -> wrist_center
    wrist_center -> EE_ori
    wrist_center -> EE_pos
    wrist_center -> EE_length
    EE_ori -> req
    EE_pos -> req
    EE_length -> dh_d7
    alt_theta_1 -> theta_1
}
    

ERROR:root:Cell magic `%%dot` not found.


In [36]:
# %%bash
# conda install pydotplus

Fetching package metadata .........
Solving package specifications: .

Package plan for installation in environment /home/robond/miniconda3/envs/RoboND:

The following NEW packages will be INSTALLED:

    pydotplus: 2.0.2-py35_0

Proceed ([y]/n)? 
pydotplus-2.0. 100% |###############################| Time: 0:00:00 480.60 kB/s


The angle theta4 is measured about Z4 on the (X4,Y4)-plane. It is equal to the angle between X4 and a vector Nxy, the projection of the v, w common normal onto the aforementioned plane.

In [ ]:
Nh = N(vector_cross_unit(v, w)) 
Vh = v.normalized()
proj_Nh_Vh = Nh.dot(Vh)
# As with gram-schmidt orthogonoalization, the xy-projection of Nh can be found by
# subtracting the projection of Nh onto Vh, which is definitionally orthogonal to X4 and Y4.
# This vector is then normalized.
Nh_xy = (Nh - (proj_Nh_Vh * Vh)).normalized()
# We need one last item to extract theta4: an x unit vector from the joint_4 reference frame,
# taken with theta4=0.
theta_4_val = N(acos(dot(Nh_xy, )))